In [ ]:
!pip install onnx
!pip install transformers

     |████████████████████████████████| 14.5MB 223kB/s 
     |████████████████████████████████| 1.9MB 4.1MB/s 
     |████████████████████████████████| 3.2MB 27.8MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bfe184c059c670c0ac45b06567957de48c837157467a4cf2192b7860b0e07c67
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import onnx
import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
## GET PYTORCH MODEL and TOKENIZER
torch_model = AutoModelForSequenceClassification.from_pretrained('neuraly/bert-base-italian-cased-sentiment')
tokenizer = AutoTokenizer.from_pretrained('neuraly/bert-base-italian-cased-sentiment')
torch_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def preprocess(tokenizer, text):
    tokens = tokenizer(text)
    input_ids = tokens['input_ids']
    while len(tokens['input_ids']) < 128:
        input_ids.append(0)
    input_ids = torch.tensor([input_ids], dtype=torch.long)
    return input_ids

In [ ]:
# see the vocab
# https://huggingface.co/neuraly/bert-base-italian-cased-sentiment/blob/main/vocab.txt

love = preprocess(tokenizer, "amo questo film!")
print(love)

hate = preprocess(tokenizer, "odio questo film!")
print(hate)

tensor([[ 102, 4714,  395, 1538, 2692,  103,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
tensor([[ 102, 9601,  395, 1538, 2692,  103,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,

In [ ]:
resultLove = torch_model(love)
print(resultLove)

resultHate = torch_model(hate)
print(resultHate)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4242,  1.4517,  1.8461]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.8452,  1.9159, -3.4431]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [ ]:
## CONVERTO TO ONNX FORMAT - AttentionMask is MISSING!!
dummy_inputs = torch.ones(1, 128, dtype=torch.int64)
torch.onnx.export(torch_model, dummy_inputs, "BERTsentiment.onnx", input_names=['input_ids'], output_names=['output'], opset_version=11)

In [ ]:
## remember the tokenizer + vocab